In [ ]:
!pip install langchain boto3
!pip install transformers
!pip install sentence_transformers

In [ ]:
pip install langchain==0.0.224

In [ ]:
pip install boto3 --upgrade

In [ ]:
import boto3
from langchain.retrievers import AmazonKendraRetriever
from typing import List
from typing import Dict
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import SagemakerEndpoint, LLMChain
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
import json

from langchain.docstore.document import Document
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

In [ ]:
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": inputs, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
#        return response_json["vectors"]
        return response_json[0]["generated_text"]



content_handler = ContentHandler()


In [119]:
from langchain import PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter


llm2 = SagemakerEndpoint(
endpoint_name="hf-llm-falcon-40b-instruct-bf16-2023-06-23-19-34-33-102",
#endpoint_name="hf-llm-falcon-7b-bf16-2023-06-24-20-08-14-262",
#endpoint_name="hf-llm-falcon-40b-bf16-2023-06-24-20-20-44-608",
model_kwargs={
     "parameters" : {"do_sample": False,
    "top_p": 0.9,
    "temperature": 0.1,
    "max_new_tokens": 200
              }},
region_name="us-east-1",
content_handler=content_handler
)

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 4000,
    chunk_overlap  = 100,
    length_function = len,
)

In [130]:
with open("bots.txt") as f:
    state_of_the_union = f.read()

docs = text_splitter.create_documents([state_of_the_union])
#print(texts)

In [131]:
from langchain.docstore.document import Document

for x in docs:
    print(x)
    print("---")

page_content='May 13th, Friday presentation to Athena\n* Vijay: AMI refresh requirement makes instance storage difficult as the hydrating of cassandra nodes from backup takes too long\n* Habib wants a dedicated session to affirmatively answer if EBS bandwidth is/is not the cause of slowness.\n* 35GBps is the aggregate ec2 network output. a fraction of that is cross region (unknown - asked Mayur to check with GNS). 40GBps is close to the Denali Stamp limit. they got a dedicated stamp in last 2 months and have not seen the issue since.\n\nMay 11th  - Summary for Athena team - (Hari, Ruchir, Shamik)\nBased on the assessment and prior discussions Athena’s key resiliency considerations fall under Observability, Operations, Testing.\n\n1. Do you have observability alerts in-place to detect network failure and ability for Athena to take action based on those alerts ?\n2. In the event of network connection failure between us-east-1 and us-west-2 what is the current application behavior ?\n3. Y

In [132]:
map_prompt_template = """Write a short sentence single line summary for following text:


{text}


Summary:"""


reduce_prompt_template = """Write a summary paragraph of the following:


{text}


Summary:"""

In [133]:
from langchain.chains.summarize import load_summarize_chain

MAP_PROMPT = PromptTemplate(template=map_prompt_template, input_variables=["text"])
REDUCE_PROMPT = PromptTemplate(template=reduce_prompt_template, input_variables=["text"])



chain = load_summarize_chain(llm2, chain_type="map_reduce", return_intermediate_steps=True, map_prompt=MAP_PROMPT,  combine_prompt=REDUCE_PROMPT)
#chain.run(docs)
chain({"input_documents": docs}, return_only_outputs=True)

{'intermediate_steps': ['\n\nThe summary for the given text is as follows:\n\n1. The presentation to Athena team discusses the key resiliency considerations for Athena, including observability, operations, and testing.\n2. The team asks questions about the current application behavior in the event of network connection failure between us-east-1 and us-west-2, the time it takes for data to be in sync, and the importance of consistency or availability of nodes.\n3. The team also asks about the observed EBS disk queue length, the tolerability of slowness in large volumes of EBS, and the use of instance storage.\n4. The team inquires about the volume and velocity of data synced by cassandra and compute layer across regions in normal scenarios, the event of restoring from backups, and the dependencies and events considered for taking backups.\n5. The team asks about the type of compute instances and EBS volumes currently leveraged, and the type of backups and price.'],
 'output_text': '\nTh